# Developing a new way to integrate ctau

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
import pandas as pd

from Likelihood import LEff
from mcmc_utils import labels_fancy
from dark_nus_utils import load_datasets
from parameters_dict import *
from const import alphaQED

In [3]:
hierarchy = 'heavy'
D_or_M = 'majorana'
distance = 'log'
smoothing_pars = (0.1, 0.1)

selection = 'cut_based'

spec = 'non_minimal'

# save_folder = f'../fig/6_limit_visualisation/{hierarchy}_{D_or_M}_{spec}/'
# !mkdir -p $save_folder

In [4]:
levels = {0.68: -2.3/2,
          0.9: -4.61/2,
          0.95: -5.99/2}

In [5]:
my_exp_analysis = load_datasets(hierarchy, D_or_M, fluxes=['FHC'], 
                                timeit=True, 
                                direct_load_objects=False,
                                load_grid=False)[f'{hierarchy}_{D_or_M}_FHC']

heavy majorana FHC
loading df base
initialising df base
Wall time: 78.41806507110596 s, CPU time: 73.483103394


In [6]:
df = my_exp_analysis.df_base

# Development of exponential integral

In [125]:
t_0_0 = (p0d_length - df['int_point', 'z'])/df['pdark_dir', 'z']
t_0_1 = (p0d_length + tpc_length - df['int_point', 'z'])/df['pdark_dir', 'z']

In [126]:
t_1_0 = (p0d_length + tpc_length + fgd_length - df['int_point', 'z'])/df['pdark_dir', 'z']
t_1_1 = (p0d_length + 2*tpc_length + fgd_length - df['int_point', 'z'])/df['pdark_dir', 'z']

In [127]:
t_2_0 = (p0d_length + 2*(tpc_length + fgd_length) - df['int_point', 'z'])/df['pdark_dir', 'z']
t_2_1 = (p0d_length + 3*tpc_length + 2*fgd_length - df['int_point', 'z'])/df['pdark_dir', 'z']

In [167]:
poe_x_min = (0 - df['int_point', 'x'])/df['pdark_dir', 'x']
poe_x_max = (p0d_dimensions[0] - df['int_point', 'x'])/df['pdark_dir', 'x']
poe_y_min = (0 - df['int_point', 'y'])/df['pdark_dir', 'y']
poe_y_max = (p0d_dimensions[1] - df['int_point', 'y'])/df['pdark_dir', 'y']

poe_s = np.stack([poe_x_min, poe_x_max, poe_y_min, poe_y_max], axis=1)
min_points = np.min(np.where(poe_s > np.atleast_2d(t_0_0.values).T, 
                             poe_s, np.inf), axis=1)
min_points[np.isinf(min_points)] = -np.inf
exp_integral_points = np.stack([t_0_0, t_0_1, t_1_0, t_1_1, t_2_0, t_2_1,], axis=-1)

which_tpc_exit = np.atleast_2d(min_points).T > exp_integral_points
exp_integral_points[~which_tpc_exit] = 0
exp_integral_points[df['pdark_dir', 'z'] <= 0] = 0
exp_integral_points[:, 1] = np.where(which_tpc_exit.sum(axis=1) == 1,
                                           min_points,
                                           exp_integral_points[:, 1])
exp_integral_points[:, 3] = np.where(which_tpc_exit.sum(axis=1) == 3,
                                           min_points,
                                           exp_integral_points[:, 3])
exp_integral_points[:, 5] = np.where(which_tpc_exit.sum(axis=1) == 5,
                                           min_points,
                                           exp_integral_points[:, 5])

for i in range(6):
    df[f'exp_integral_points_{i}'] = exp_integral_points[:,i]

In [129]:
exp_integral_points

array([[242.24374133, 343.61541057, 380.61606984, 481.98773907,
        518.98839834, 620.36006757],
       [205.03758801, 310.22431006, 348.61746361, 408.27681885,
          0.        ,   0.        ],
       [206.45772817, 309.81874325, 347.54551376, 450.90652884,
        488.63329934, 591.99431442],
       ...,
       [ 31.89952707, 145.6209556 , 187.12927701, 300.85070554,
          0.        ,   0.        ],
       [ 13.30836312, 114.44957368, 151.36611553, 252.50732609,
        289.42386795, 358.60842739],
       [230.54817266, 331.65002693, 368.55220373, 469.654058  ,
          0.        ,   0.        ]])

In [168]:
def compute_ctau_weights(df, ctau):
    scale = df['betagamma']*ctau
    return np.exp(-df[f'exp_integral_points_0']/scale) - np.exp(-df[f'exp_integral_points_1']/scale) +\
           np.exp(-df[f'exp_integral_points_2']/scale) - np.exp(-df[f'exp_integral_points_3']/scale) +\
           np.exp(-df[f'exp_integral_points_4']/scale) - np.exp(-df[f'exp_integral_points_5']/scale)

In [172]:
out = compute_ctau_weights(df, ctau=3)